Author: Quim Serra Faber

Date: December 2020

# 1 Introduction

In this project, the focus is on using neighbourhood statistics as predictors of alcohol and tobacco consumption. The data sets are:

- [Wijk en Buurtstatistieken](https://www.cbs.nl/nl-nl/reeksen/kerncijfers-wijken-en-buurten-2004-2020)

- [Rokers per wijk](https://www.volksgezondheidenzorg.info/onderwerp/roken/regionaal-internationaal/regionaal#node-rokers-wijk)

- [Alcoholgebruik volgens richtlijn per wijk](https://www.volksgezondheidenzorg.info/onderwerp/alcoholgebruik/regionaal-internationaal/regionaal#node-alcoholgebruik-volgens-richtlijn-ggd-regio)


All of the used datasets are from 2016, since it is the most recent year in which all the datasets are available.

The latest two datasets have the same structure. The first one is more detailed, and for example has information not only for each "wijk" but also for every "buurt" inside of each "wijk". This more detailed data is not present in the other two datasets, so it will have to be ignored. All the comparisons will be done with respect to the "wijks". 

# 2 Data Processing

## 2.1 Loading Data

In [1]:
# Load capabilities
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Immediately show plots
%matplotlib inline

# Import the data into a pandas data frame, in all cases use a multiindex with "gemeente" and "wijk"
dneigh = pd.read_excel('kwb-2016.xls', index_col= [3,2], decimal=",")
dsmoke = pd.read_csv('smap2016vzinfo_roker.csv', index_col=[1,2], sep=';', decimal=",")
ddrink = pd.read_csv('smap2016vzinfo_richtlijn_alcohol.csv', index_col=[1,2], sep=';', decimal=",")

## 2.2 Inspecting Data

In [2]:
# Inspect the neighbourhood data by calling some summary functions
print("Data information:")
dneigh.info()
print("Data description:")
print(dneigh.describe())

Data information:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 16194 entries, ('Nederland', 'Nederland') to ('Menterwolde', 'Verspreide huizen Muntendam')
Columns: 110 entries, gwb_code_10 to g_gewsek
dtypes: int64(40), object(70)
memory usage: 13.8+ MB
Data description:
         gwb_code_8         a_inw         a_man       a_vrouw       a_00_14  \
count  1.619400e+04  1.619400e+04  1.619400e+04  1.619400e+04  1.619400e+04   
mean   5.878220e+06  4.192644e+03  2.077820e+03  2.113573e+03  6.914600e+02   
std    6.198315e+06  1.340222e+05  6.643739e+04  6.758498e+04  2.209844e+04   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    7.003012e+05  2.300000e+02  1.200000e+02  1.100000e+02  3.000000e+01   
50%    3.639202e+06  9.650000e+02  4.800000e+02  4.800000e+02  1.450000e+02   
75%    8.281277e+06  2.570000e+03  1.275000e+03  1.295000e+03  4.200000e+02   
max    1.987031e+07  1.697912e+07  8.417135e+06  8.561985e+06  2.799772e+06   

           

In [3]:
# Inspect the smoking data by calling some summary functions
print("Data information:")
dsmoke.info()
print("Data description:")
print(dsmoke.describe())

Data information:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3351 entries, ("'s-Gravenhage", 'Gemeente') to ('Zwolle', 'Wijk 52 Soestweteringlanden')
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          3351 non-null   int64  
 1   Rokers (%)  2948 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 90.4+ KB
Data description:
                id   Rokers (%)
count  3351.000000  2948.000000
mean    778.064160    20.168250
std     610.567152     4.293861
min       3.000000     9.000000
25%     307.000000    17.000000
50%     588.000000    19.000000
75%     995.000000    22.000000
max    1987.000000    53.000000


In [4]:
# Inspect the drinking data by calling some summary functions
print("Data information:")
ddrink.info()
print("Data description:")
print(ddrink.describe())

Data information:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3351 entries, ("'s-Gravenhage", 'Gemeente') to ('Zwolle', 'Wijk 52 Soestweteringlanden')
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3351 non-null   int64  
 1   Alcoholrichtlijn (%)  2948 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 90.4+ KB
Data description:
                id  Alcoholrichtlijn (%)
count  3351.000000           2948.000000
mean    778.064160             39.298847
std     610.567152              6.443029
min       3.000000             23.000000
25%     307.000000             35.000000
50%     588.000000             39.000000
75%     995.000000             43.000000
max    1987.000000             75.000000


## 2.3 Combining Data

Now that we have three different dataframes we have to make them cohesive, since the neighbourhood dataframe has a different order of "wijks", and many more samples. We start by putting the indexes in the same order:

The first problem to be solved is the difference in number of samples: for that we must eliminate the additional rows of "buurts" in the neighbourhood dataframe, and also eliminate rows which do not add additional information such as the "gemeentes" in both the smoke and drink dataframes.

In [5]:
# Only maintain the rows corresponding to "Wijks"
dneigh = dneigh[(dneigh['recs'] == 'Wijk')]
len(dneigh.index)

2981

In [6]:
# Iteratively remove the empty rows
for i in range(len(dsmoke.index)):
    try:
        if dsmoke.index[i][1] == 'Gemeente':
            dsmoke.drop(dsmoke.index[i], inplace=True)
    except IndexError:
        break  
# Note the use of try and except, since when dropping rows the length of the dataframe gets reduced
for i in range(len(ddrink.index)):
    try:
        if ddrink.index[i][1] == 'Gemeente':
            ddrink.drop(ddrink.index[i], inplace=True)
    except IndexError:
        break
print(len(dsmoke.index))

# Although dropping a row inside a for loop can lead to problems (the row after the dropped row will be skipped), 
# in this case it is not a problem: due to the nature of the datasets, there are no consecutive 'Gemeente' rows.

2961


We thus see that there is a difference of 20 rows, and the case is that there are some additional "wijks" in the neighbourhood dataframe. Upon inspection, we can see that these additional rows are just full of empty data, so if we delete them we will not miss any information, and the dataframes will be cohesive. In this case, we just compare the indexes, and if they are different we drop them:

In [7]:
# Two iterations, to make sure that we do not skip any consecutive index
for e in range(0,20):
    for i in range(len(dneigh.index)):
        try:
            if dneigh.index[i][1] != ddrink.index[i][1]:
                dneigh.drop(dneigh.index[i], inplace=True)
                break
        except IndexError:
            break
print(len(dneigh.index))

2961


The number of rows is the same in all dataframes, so now we can proceed to sort them. The easiest way to do it is alphabetically:

In [8]:
# Sort all dataframes so that the locations are in the same order
dneigh = dneigh.sort_index(ascending=True)
dsmoke = dsmoke.sort_index(ascending=True)
ddrink = ddrink.sort_index(ascending=True)
dneigh.drop(dneigh.index[0], inplace=True)

Nevertheless, this sorting does not entirely work, because in all dataframes there are some rows with indexes that do not exist in the other dataframes. As a result, at some points of the dataframes the rows are mismatched. 
This is solved by dropping all the rows with indexes that are missing in other dataframes:

In [9]:
# Remove all rows with indexes that exist in dneigh but not in dsmoke
for i in dneigh.index: #  Iterate over the indexes of dneigh
    try:
        dsmoke.loc[i] # Compute the value of dsmoke with the dneigh index
    except:
        dneigh.drop([i], axis=0, inplace = True) #  In case of error drop that row

Proceed with the same procedure for dsmoke (note that it is not necessary to do it again with the ddrink dataframe, since both dataframes share the same rows):

In [10]:
# Remove all rows with indexes that exist in dsmoke but not in dneigh
for i in dsmoke.index: #  Iterate over the indexes of dsmoke
    try:
        dneigh.loc[i] # Compute the value of dneigh with the dsmoke index
    except:
        dsmoke.drop([i], axis=0, inplace = True) #  In case of error drop that row
        ddrink.drop([i], axis=0, inplace = True)

Another problem to solve is the fact that both 'Rokers (%)' and 'Alcoholrichtlijn (%)' columns have some missing data. In this case, it is quite pointless to impute new data, since these are the target variables. Therefore, we remove the rows containing this missing data in all dataframes:

In [11]:
# Two iterations, to make sure that we do not skip any consecutive index
for e in range(1, 20):
    for i in range(len(dsmoke.index)): 
        try:
            if np.isnan(dsmoke['Rokers (%)'][i]): #  Condition that there is NaN
                dneigh.drop(dneigh.index[i], inplace=True) #  Drop row
                dsmoke.drop(dsmoke.index[i], inplace=True)
        except IndexError:
            break  

In [12]:
# Same algorithm for ddrink
for e in range(1, 20):
    for i in range(len(ddrink.index)):
        try:
            if np.isnan(ddrink['Alcoholrichtlijn (%)'][i]):
                ddrink.drop(ddrink.index[i], inplace=True)
        except IndexError:
            break  

Check that this has worked properly with the following for loop, which compares the indices of dsmoke and dneigh row by row:

In [13]:
for i in range(len(dsmoke.index)):
    if dsmoke.index[i][1] != dneigh.index[i][1]:
        print(dsmoke.index[i][1])

No index is printed so the rows of the dataframes are succesfully synchronized. 

## 2.4 Splitting Data

It is necessary to split the data for the Machine Learning models:

In [14]:
# Create target variables and rest of the datset
X = dneigh.loc[:, 'a_inw':'g_gewsek']
ys = dsmoke['Rokers (%)'].values
yd = ddrink['Alcoholrichtlijn (%)'].values

# Test if the separation makes sense:
print("X.shape: {} ys.Shape: {}".format(X.shape, ys.shape))
print("X.shape: {} yd.Shape: {}".format(X.shape, yd.shape))

X.shape: (2916, 105) ys.Shape: (2916,)
X.shape: (2916, 105) yd.Shape: (2916,)


Some values of X are NaN, or some string, which we are not interested in. First, we convert all points into floats, and in case that they are a string, NaN. 

In [15]:
# Convert all values to floats
X = X.apply(pd.to_numeric, errors='coerce')

There are a few columns that are full of NaNs, so we drop them:

In [17]:
# Two iterations, to make sure that we do not skip any index
for i in range(len(list(X))):
    for e in list(X): #  Iterate over columns
        if X[e].isna().sum() == len(X.index): #  Check the count of NaNs in the column
            X.drop([e], inplace=True, axis = 1) #  Drop the column
print("X.shape: {} ys.Shape: {}".format(X.shape, ys.shape))

X.shape: (2916, 103) ys.Shape: (2916,)


So two columns have been removed.

We can also check that the target variables have no NaN variables:

In [18]:
print(sum(np.isnan(ys)))
print(sum(np.isnan(yd)))

0
0


Nevertheless, that is not the case for X:

In [19]:
X.isna().any()

a_inw       False
a_man       False
a_vrouw     False
a_00_14     False
a_15_24     False
            ...  
ste_mvs     False
ste_oad     False
g_wodief     True
g_vernoo     True
g_gewsek     True
Length: 103, dtype: bool

In this case, we use a simple imputer:

In [20]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(X)
X = imp.transform(X)

Now we can finally split the data:

In [21]:
# Split data
X_train, X_test, ys_train, ys_test = train_test_split(X, ys, test_size=0.3, random_state=0)
X_train, X_test, yd_train, yd_test = train_test_split(X, yd, test_size=0.3, random_state=0)

Finally, the data is not very sparse and the number of features is low in comparison to the number of samples. This means that ensemble methods such as random forests or gradient boosted decision trees will likely perform better than linear models, and therefore scaling is probably not needed.

Additionally, all the features are numerical, so there is no need to use one-hot-encoding. This also means that we will use regression models, and the used metric as an indicative of performance will be R^2.